# Hydroelastic Contact: Basics
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.

Drake에 익숙하지 않다면 [index](./index.ipynb)를 참조하거나 [authoring_multibody_simulation.ipynb](./authoring_multibody_simulation.ipynb)을 공부하자. 또한 Hydroelastic Contact 사용자 가이드는 [여기](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)에서 더 많은 정보를 찾을 수 있다.

## 소개(Introduction)

이 튜토리얼은 유체-탄성 접촉(hydroelastic contacts)을 포함하는 동적 시뮬레이션을 설정하는 방법을 보여준다. 또한, 접촉 결과를 수치적으로 보고하고 MeshCat에서 시각화하는 방법도 보여준다. 예시로 유연한 블록이 단단한 직사각형 판 위에 떨어지는 간단한 예를 사용한다. 접촉력을 계산하고 시각화한다.

예를 들어, 왼쪽 아래 그림은 블록의 접촉 한 프레임을 보여준다. 복잡한 접촉력 집합은 접촉 wrench(힘/토크)로 시각화되며, 힘은 빨간색 화살표로, 토크는 파란색 화살표로 표시한다. 오른쪽 아래 그림은 유체-탄성 접촉 표면을 확대하여 압력 분포를 보여준다. 검정은 압력이 없는 위치(오른쪽 가장)를 나타내고, 흰색은 가장 높은 압력(왼쪽 가장)을 나타낸다. 블록이 판 위에서 앞뒤로 흔들리고 있기 때문에 접촉 패치(contact patch)는 전체 직사각형면이 아닌 블록 바닥면의 한 모서리에 위치한다.

![HydroelasticTutorialImage600x388.jpg](https://drake.mit.edu/doxygen_cxx/HydroelasticTutorialImage600x388.jpg)

Drake의 *수력탄성 접촉 모델(Hydroelastic Contact Model)*은 Ryan Elandt의 압력장 접촉 [*Pressure Field Contact (PFC)*](https://ecommons.cornell.edu/handle/1813/112919) 모델 발명과 함께 도요타 연구소에서 발전했다. Ryan의 박사 학위 논문은 PFC 모델의 유용성을 설명한다:

> "*...이 모델은 접촉 영역에서 변형이 질적으로 중요하지만 정확하게 계산할 필요가 없는 상황에서 유용하다. PFC는 역학의 근사 모델(convergent approximation)이 아니라 연속체(continuum mechanics) 모델이다. PFC에서 두 접촉 물체 각각은 변할 수 없는 내부 가상 압력장을 가지고 있다. ... PFC 접촉 표면은 두 압력장이 같은 점들의 집합으로 정의된다.  ...*"

수력탄성 접촉(hydroelastic contact)에 대한 훌륭한 개요를 얻으려면 두 개의 기사를 읽어보자 (각각 10분 미만):

- [로봇 작업을 위한 접촉 시뮬레이션 재고(Rethinking Contact Simulation for Robot Manipulation)](https://medium.com/toyotaresearch/rethinking-contact-simulation-for-robot-manipulation-434a56b5ec88)
- [Drake: 로봇공학과 머신러닝 시대의 모델 기반 설계(Model-based design in the age of robotics and machine learning.)](https://medium.com/toyotaresearch/drake-model-based-design-in-the-age-of-robotics-and-machine-learning-59938c985515)

일반적으로 수력탄성 접촉은 유한 요소 해석(Finite Element Analysis (FEA))보다 빠르고 점 접촉 모델보다 더 연속적이고 안정적이며, 특히 마찰, 굴림, 에너지 소산(energy dissipation), 비 볼록 지형(non-convex geometries) 등의 실제 상황에서 효과적이다. 수력탄성 접촉은 변형 (FEA에서와 같이)을 포함하지 않으며 일부 시뮬레이터 (Drake 내 옵션 중 하나 포함)에서 사용되는 점 접촉 모델보다 느리다.

두 강체가 수력탄성 접촉으로 인해 상호 작용할 때 어느 쪽도 실제 기계적 변형을 겪지 않는다. 즉, 물체의 질량, 질량 중심(center of mass), 관성 특성(inertia properties)은 수력탄성 접촉에 영향을 받지 않는다. 하지만 힘 상호 작용은 마치 변형이 있었던 것처럼 계산된다.

### Compliant-hydroelastic geometries and rigid-hydroelastic geometries

수력탄성 형상(hydroelastic geometries)에는 2가지 종류가 있다.
첫 번째, "순응형 수력탄성 (compliant-hydroelastic)"은 내부 압력장을 표현하는 사면체 메쉬(tetrahedral mesh)를 포함하는 고체이다. (참고: 고체 표면의 압력은 0이며, 빠른 계산을 위해 시뮬레이터는 접촉이 가능하기 전에 내부 압력장을 계산한다.) 두 번째, "강체 수력탄성 (rigid-hydroelastic)"은 삼각형 표면 메쉬(triangle surface mesh)만 가지고 있으며 (따라서 고체 또는 표면일 수 있음) 무한히 단단하다고 간주한다.

![HydroelasticTutorialCompliantRigidOutsideInside800x669.jpg](https://drake.mit.edu/doxygen_cxx/HydroelasticTutorialCompliantRigidOutsideInside800x669.jpg)

### 접촉면(Contact surface)

아래 그림은 순응형 수력탄성 실린더와 순응형 수력탄성 구체(sphere) 사이의 접촉을 보여준다. 접촉 표면은 두 고체 내부에 존재하며 두 압력장이 동일한 표면으로 정의한다. 접촉 표면의 음영은 검정 (압력 없음)에서 빨강 (높은 압력)으로 변화하는 압력 분포를 나타낸다. 수학적으로, 접촉 표면은 실린더 표면과 구체 표면의 교차 부분 사이의 혼합 표면이다. 개념적으로, 크기가 비슷한 두 구체의 경우 접촉 표면은 유수탄성 계수가 더 큰 구체의 표면에 더 가까워진다. 수력탄성 접촉 사용자 안내의 [이 섹션](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html#hug_hydro_theory)을 참고하자.

![HydroelasticTutorialContactSurfaceCompliantCompliant.png](https://drake.mit.edu/doxygen_cxx/HydroelasticTutorialContactSurfaceCompliantCompliant.png)

강체 수력탄성 물체(rigid-hydroelastic geometry)는 무한히 단단하다고 가정되기 때문에 순응형 수력탄성 물체와 강체 수력탄성 물체 사이의 접촉 표면은 항상 강체 수력탄성 물체의 표면에 위치한다. 아래 그림과 같이 두 개의 순응형 수력탄성 타원체 거품 손가락(bubble fingers)이 강체 수력탄성인 막대 모양 주걱 손잡이(spatula handle)와 접촉하고 있다. 따라서 접촉 표면은 주걱 손잡이(spatula handle)의 표면에 있다.

![HydroelasticTutorialContactSurfaceRigidCompliantBubble.png](https://drake.mit.edu/doxygen_cxx/HydroelasticTutorialContactSurfaceRigidCompliantBubble.png)

### Contact mode between compliant-hydroelastic geometries and rigid-hydroelastic geometries

일반적으로 물체의 강성을 필요에 따라 고무처럼 유연하게 하거나 강철처럼 단단하게 설정할 수 있는 순응형 수력탄성 물체(compliant-hydroelastic geometries)를 사용하는 것이 좋다. 순응형 수력탄성 물체는 사면체 메쉬(tetrahedral meshes)를 필요로 한다. Drake는 구체, 타원체, 실린더, 캡슐, 박스와 같은 기본 도형 및 볼록 표면 메쉬(convex surface mesh)를 가진 고체에 대해 순응형 수력탄성 물체에 필요한 사면체 메쉬를 자동으로 생성할 수 있다. 볼록하지 않은 물체(non-convex objects)의 경우 순응형 수력탄성으로 사용하려면 사면체 메쉬를 포함한 VTK 파일 (obj 파일은 아님)을 반드시 제공해야만 한다.

순응형 수력탄성 물체는 다른 순응형 수력탄성 물체 또는 강체 수력탄성 물체와 잘 상호작용한다.

반면에 강체 수력탄성 물체는 쉽게 사면체 부피 메쉬를 생성할 수 없는 볼록하지 않은 표면 메쉬 (예: obj 파일)를 가진 물체를 나타내는데 중요하게 사용된다. 현재 강체 수력탄성 물체는 순응형 수력탄성 물체와만 상호작용할 수 있다. 다른 강체 수력탄성 물체와 상호작용하면 시뮬레이터는 점 접촉으로 전환된다. 아래 표는 이런저런 경우에 대한 자세한 정보를 제공한다.


|                                     | rigid-hydroelastic geometry   | compliant-hydroelastic geometry |
|:------------------------------------|:------------------------------|:--------------------------------|
| **rigid-hydroelastic geometry**     | point contact                 | hydroelastic contact            |
| **compliant-hydroelastic geometry** | hydroelastic contact          | hydroelastic contact            |
| **non-hydroelastic geometry**       | point contact                 | point contact                   |



## Outline

이 튜토리얼은 다음과 같은 작업 방법을 보여준다.:

1. 순응형 수력탄성 상자를 위한 SDFormat 문자열 정의

    * 시각적 geometry 정의
    * 수력탄성 속성을 포함한 충돌 지오메트리 정의
    * 질량 및 관성 특성(inertia properties) 정의


2. 강체 수력탄성 box를 위한 SDFormat 문자열 정의

3. 두 box를 사용하여 MultibodyPlant 다이어그램 생성하기

4. 시뮬레이션 시각화

5. LeafSystem 생성: 시뮬레이션 마지막에 접촉 결과를 수치적으로 보고

6. 접촉 결과를 시각화하기 위해서 ContactVisualizer 추가


## MeshCat 시작하기(Start MeshCat)

튜터리얼에서 [모델 보기(Viewing models)](./authoring_multibody_simulation.ipynb#Viewing-models) 섹션과 MeshCat 소개는 [Authoring a Multibody Simulation](./authoring_multibody_simulation.ipynb)를 살펴보자.

In [ ]:
from pydrake.geometry import StartMeshcat

# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

## 순응형 수력탄성 box 생성하기(Create compliant-hydroelastic box)

*브라우져에 MeshCat 브라우저 탭이 열려있는지 확인한다. 위에 링크를 클릭하여 열 수 있습니다.*

순응형 수력탄성 box를 SDFormat 문자열로 생성하고 시각화해볼 것이다.

`ModelVisualizer`를 사용하여 SDFormat 파일을 검증한다.
이 도구는 MeshCat에게 box를 표시하도록 지시한다.

### Visual geometry

먼저 10cm x 20cm x 40cm 크기의 상자 시각 형태를 만들어보자. MeshCat에서 빨간색 X축, 초록색 Y축, 파란색 Z축을 확인한다. `<size>0.10 0.20 0.40</size>` 태그는 상자의 크기를 미터 단위로 지정한다. `<diffuse>` 태그는 0부터 1 사이의 RGB 및 알파 값을 지정하며 이에 따라 색상과 투명도를 설정할 수 있다.

In [ ]:
from pydrake.visualization import ModelVisualizer

visual_box_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="CompliantBox">
    <pose>0 0 0 0 0 0</pose>
    <link name="compliant_box">
      <visual name="visual">
        <geometry>
          <box>
            <size>0.10 0.20 0.40</size>
          </box>
        </geometry>
        <material>
          <diffuse>1.0 1.0 1.0 0.5</diffuse>
        </material>
      </visual>
    </link>
  </model>
</sdf>
"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
visualizer.parser().AddModelsFromString(visual_box_sdf, "sdf")
visualizer.Run(loop_once=True)

### Collision geometry with hydroelastic properties

이제 수력탄성 속성에 `<collision>` 블록을 추가해보자. `<collision>` 태그가 없으면 시뮬레이션내에서 물체 간 접촉이 발생하지 않는다.

시각 형태와 동일하게 10x20x40 센티미터 크기의 상자를 충돌 형태로 지정한다.

Drake는 충돌을 나타내는 용어로 근접성(proximity)이라는 용어도 사용합니다. `<drake:proximity_properties>` 블록은 이러한 수력탄성 충돌을 제어한다.:

        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1e7</drake:hydroelastic_modulus>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
        </drake:proximity_properties>

`<drake:compliant_hydroelastic/>` 태그를 사용하여 순응형 수력탄성 객체로 지정한다.

`<drake:hydroelastic_modulus>` 값을 1e7 파스칼로 설정한다. 이는 약 1GPa 영률(Young's modulus)의 고밀도 폴리에틸렌 (HDPE)과 비슷한 수준이다. 수력탄성 계수(Hydroelastic modulus)는 물리적 특성이 아니라 접촉 모델을 조정하는 수치 변수이다. 일반적으로 수력탄성 계수는 영률(Young's modulus)의 약 1/100로 설정할 수 있다.

`<drake:mu_dynamic>` (단위 없음) 값을 0.5로 설정하여 동적 (즉, 운동) 마찰 계수를 정의한다.

`<drake:hunt_crossley_dissipation>` 값을 1.25 초/미터로 설정하여 Hunt & Crossley 모델의 감쇠 상수를 지정한다.

In [ ]:
from pydrake.visualization import ModelVisualizer

collision_box_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="CompliantBox">
    <pose>0 0 0 0 0 0</pose>
    <link name="compliant_box">
      <collision name="collision">
        <geometry>
          <box>
            <size>0.10 0.20 0.40</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1e7</drake:hydroelastic_modulus>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
      </collision>
      <visual name="visual">
        <geometry>
          <box>
            <size>0.10 0.20 0.40</size>
          </box>
        </geometry>
        <material>
          <diffuse>1.0 1.0 1.0 0.5</diffuse>
        </material>
      </visual>
    </link>
  </model>
</sdf>
"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
visualizer.parser().AddModelsFromString(collision_box_sdf, "sdf")
visualizer.Run(loop_once=True)

### Inertial properties

동역학(dynamics)을 시뮬레이션하기 위해서 질량 `<mass>` 1kg과 관성 모멘트 행렬 `<inertia>`를 추가한다.

질량 중심점에 대한 상자의 `<inertia>`는 아래와 같이 계산한다:

    Ixx = m(s.y² + s.z²)/12
    Iyy = m(s.x² + s.z²)/12
    Izz = m(s.x² + s.y²)/12

여기서 s.x, s.y, s.z는 상자의 크기를 나타낸다.

실제로는 아래 코드처럼 `SpatialInertia.SolidBoxWithMass()` 함수를 사용하여 관성 모멘트 행렬을 계산하고, 결과 값을 SDFormat 문자열에 복사할 수 있다. 6x6 행렬에 대한 자세한 설명은 [SpatialInertia](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_spatial_inertia.html) 문서를 참조하자.

In [ ]:
from pydrake.multibody.tree import SpatialInertia

spatial_inertia = SpatialInertia.SolidBoxWithMass(
          mass=1, lx=0.1, ly=0.2, lz=0.4)
mass = spatial_inertia.get_mass()
center_of_mass = spatial_inertia.get_com()
matrix6 = spatial_inertia.CopyToFullMatrix6()

print(f"mass = {mass}\n")
print(f"p_PScm = center of mass = {center_of_mass}\n")
print(f"I_SP = rotational inertia = \n{matrix6[:3, :3]}\n")

아래의 `compliant_box()` 함수는 주어진 `hydroelastic_modulus` 값을 기반으로 순응형 수력탄성 상자의 SDFormat을 반환한다. 이 함수는 나중에 수력탄성 계수(hydroelastic modulus)의 다양한 값을 실험할 때 사용하게 된다. `<inertial>` 블록의 숫자들은 앞서 SpatialInertia를 사용하여 계산한 결과를 복사한 것이다.

파라미터 `hydroelastic_modulus`는 문자열 `'1e7'` (작은따옴표 `'`로 묶임)으로 지정되며, 부동 소수점 숫자 `1e7`가 아니다. 이는 SDFormat에서 형식 지정으로 인해 정밀도를 잃지 않도록 하기 위해서다.

In [ ]:
from pydrake.visualization import ModelVisualizer

# Define a compliant-hydroelastic box from
# a given hydroelastic modulus.
def compliant_box(hydroelastic_modulus='1e7'):
    return f"""<?xml version="1.0"?>
    <sdf version="1.7">
      <model name="CompliantBox">
        <pose>0 0 0 0 0 0</pose>
        <link name="compliant_box">
          <inertial>
            <mass>1.0</mass>
            <inertia>
              <ixx>0.016666</ixx> <ixy>0.0</ixy> <ixz>0.0</ixz>
              <iyy>0.014166</iyy> <iyz>0.0</iyz>
              <izz>0.004166</izz>
            </inertia>
          </inertial>
          <collision name="collision">
            <geometry>
              <box>
                <size>0.10 0.20 0.40</size>
              </box>
            </geometry>
            <drake:proximity_properties>
              <drake:compliant_hydroelastic/>
              <drake:hydroelastic_modulus>{hydroelastic_modulus}</drake:hydroelastic_modulus>
              <drake:mu_dynamic>0.5</drake:mu_dynamic>
              <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
            </drake:proximity_properties>
          </collision>
          <visual name="visual">
            <geometry>
              <box>
                <size>0.10 0.20 0.40</size>
              </box>
            </geometry>
            <material>
              <diffuse>1.0 1.0 1.0 0.5</diffuse>
            </material>
          </visual>
        </link>
      </model>
    </sdf>
    """

compliant_box_sdf = compliant_box()
print(compliant_box_sdf)

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
visualizer.parser().AddModelsFromString(compliant_box_sdf, "sdf")
visualizer.Run(loop_once=True)

## rigid-hydroelastic box 생성하기(Create rigid-hydroelastic box)

다음 SDFormat 문자열은 수력탄성 상자를 지정한다. 이는 순응형 수력탄성 상자와 유사한 형태다.

`<visual>`과 `<collision>` 지오메트리는 모두 크기가 60cm x 1m x 5cm인 상자이다. MeshCat에서 빨간색 X축, 초록색 Y축, 파란색 Z축을 확인하자.

`<drake:rigid_hydroelastic/>` 태그를 사용하여 강체 수력탄성 지오메트리를 지정한다.

이 태그는 `<drake:hydroelastic_modulus>`를 사용하지 않으며, 본질적으로 무한한 수력탄성 계수를 가진다.
이 강체 수력탄성 상자는 이전의 순응형 수력탄성 상자와 동일한 마찰 계수와 감쇠 상수(dissipation constants)를 사용한다.

다음 섹션에서 `Diagram`을 설정할 때 강체 상자를 world 프레임에 고정할 것이므로 강체 상자의 `<mass>`와 `<inertia>`를 지정하지 않는다. 따라서 움직이지 않는다.

`<frame name="top_surface">`는 상자 상단에 있는 프레임이며 다음 태그를 포함한다:

    <pose relative_to="rigid_box_link">0 0 0.025 0 0 0</pose>

이 프레임은 박스 중앙 위 2.5cm에 있으며 이 위치는 박스 높이의 절반이다. 다음 섹션에서 world 좌표계의 원점에 `top_surface` 프레임 위치시키는 scene을 생성할 것이다.

In [ ]:
from pydrake.visualization import ModelVisualizer

# Create a rigid-hydroelastic table top
rigid_box_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="RigidBox">
    <link name="rigid_box_link">
      <visual name="visual">
        <pose>0 0 0 0 0 0</pose>
        <geometry>
          <box>
            <size>0.6 1.0 0.05</size>
          </box>
        </geometry>
        <material>
         <diffuse>0.9 0.8 0.7 0.5</diffuse>
        </material>
      </visual>
      <collision name="collision">
        <pose>0 0 0 0 0 0</pose>
        <geometry>
          <box>
            <size>0.6 1.0 0.05</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:rigid_hydroelastic/>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
        </drake:proximity_properties>
      </collision>
    </link>
    <frame name="top_surface">
      <pose relative_to="rigid_box_link">0 0 0.025 0 0 0</pose>
    </frame>
  </model>
</sdf>
"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat, visualize_frames=True)
visualizer.parser().AddModelsFromString(rigid_box_sdf, "sdf")
visualizer.Run(loop_once=True)

## scene의 Diagram 생성하기(Create Diagram of the scene)

아래 `add_scene()` 함수는 우리가 만든 2개 박스를 사용하여 scene을 만든다.

`DiagramBuilder`를 사용하여 `MultibodyPlant`와 `SceneGraph`를 만든다.
`Parser`를 사용하여 2개 박스의 2개 SDFormat 문자열을 scene의 `Diagram`에 추가한다.

강한 충격 시 안정성을 위해 `discrete_contact_approximation`을 "similar"로 설정한다. [DiscreteContactApproximation](https://drake.mit.edu/doxygen_cxx/namespacedrake_1_1multibody.html#:~:text=enum%20DiscreteContactApproximation) 문서 참조.

`WeldFrames()` 호출을 통해 강체 상자의 상단 표면을 world 프레임에 고정한다.

순응형 박스를 강체 박스 위 1m 높이에 위치시킨다.

이 단계 이후에, 다음 섹션에서 `DiagramBuilder`에 시각화를 추가할 것이다.

In [ ]:
from pydrake.math import RigidTransform
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlant, MultibodyPlantConfig
from pydrake.systems.framework import DiagramBuilder

def clear_meshcat():
    # Clear MeshCat window from the previous blocks.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

def add_scene(time_step=1e-3):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlant(
        MultibodyPlantConfig(
            time_step=time_step,
            discrete_contact_approximation="similar"),
        builder)
    parser = Parser(plant)

    # Load the table top and the box we created.
    parser.AddModelsFromString(compliant_box_sdf, "sdf")
    parser.AddModelsFromString(rigid_box_sdf, "sdf")

    # Weld the rigid box to the world so that it's fixed during simulation.
    # The top surface passes the world's origin.
    plant.WeldFrames(plant.world_frame(), 
                     plant.GetFrameByName("top_surface"))

    # Finalize the plant after loading the scene.
    plant.Finalize()

    # Set how high the center of the compliant box is from the world's origin. 
    # W = the world's frame
    # C = frame at the center of the compliant box
    X_WC = RigidTransform(p=[0, 0, 1])
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName("compliant_box"), X_WC)

    return builder, plant

## 시뮬레이션의 시각화 설정하기(Set up visualization of the simulation)

앞서 살펴본 `add_scene()` 함수에는 시각화 기능이 포함되지 않았다. 이를 위해 아래의 `add_viz()` 함수를 사용한다.

`add_scene()`에서 `DiagramBuilder`를 사용한다. `meshcat` 인터페이스는 이미 사용 중인 MeshCat 창을 대표한다.

시뮬레이션에서 초당 256개의 프레임을 게시하도록 `publish_period`를 1/256로 설정한다. 이 예제에서는 접촉 변화를 더 자세히 관찰하기 위해 자주 publish하도록 설정한다. 자세한 내용은 [VisualizationConfig](https://drake.mit.edu/doxygen_cxx/structdrake_1_1visualization_1_1_visualization_config.html) 문서를 참조하자.

현재 단계에서는 `publish_contacts`를 False로 설정하며, 이 튜토리얼 후반부에 가서야 접촉 시각화를 설정할 것이다.

0초 동안 시뮬레이션을 실행하여 `Diagram` 생성을 테스트한다. 강체 상자 위 1m 높이에 있는 순응형 박스가 나타난다. 동시에 2 박스를 보려면 화면을 축소해야 할 수도 있다.

In [ ]:
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig

def add_viz(builder, plant):
    ApplyVisualizationConfig(
        config=VisualizationConfig(
                   publish_period = 1 / 256.0,
                   publish_contacts = False),
        builder=builder, meshcat=meshcat)
    
    return builder, plant


from pydrake.systems.analysis import Simulator

# Test creation of the diagram by simulating for 0 second.
# For now, use only the DiagramBuilder from the first return value and
# ignore the other return value. We will use it later.
clear_meshcat()
builder, plant = add_scene()
add_viz(builder, plant)
simulator = Simulator(builder.Build())
simulator.AdvanceTo(0)

### 시뮬레이션 실행하기(Run the simulation)

아래 `run_simulation()` 함수를 정의한다. 이전 단계에서 사용한 `DiagramBuilder`를 통해 `Diagram`을 생성한다. `Diagram`을 Simulator에 전달한다. 시뮬레이션 결과를 `meshcat`에 저장하여 나중에 재생하는데 사용한다. 순응형 박스가 떨어지면서 단단한 박스와 접촉하는 것을 볼 수 있다.

In [ ]:
from pydrake.systems.analysis import Simulator

def run_simulation(sim_time, time_step=1e-3):
    clear_meshcat()
    
    builder, plant = add_scene(time_step)
    add_viz(builder, plant)
    
    diagram = builder.Build()
    
    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.)
    
    meshcat.StartRecording(frames_per_second=256.0)
    simulator.AdvanceTo(sim_time)
    meshcat.StopRecording()


run_simulation(sim_time=6)

### 시뮬레이션 녹화한 것을 재생하기(Playback recording of the simulation)

다음 명령 `meshcat.PublishRecording()`은 MeshCat 탭에 재생을 생성한다. MeshCat 탭의 `Animations` 패널을 활성화한다.

In [ ]:
meshcat.PublishRecording()

MeshCat 창에서 Open Controls를 클릭하자. `Animations` 패널이 나타난다.

**play** 버튼을 사용하여 애니메이션을 재생한다.

**timeScale**을 조절하여 애니메이션 속도를 변경한다. 접촉 변화를 좀 더 자세히 관찰하기 위해서는 0.1로 설정하여 느리게 재생하는 것이 좋다.

**pause**버튼을 사용하여 애니메이션을 중간에 멈추고, **reset** 버튼을 사용하여 애니메이션을 time 0으로 처음으로 옮길 수 있다.


재생하는 동안 순응형 박스가 아래로 떨어져 강체 상자와 접촉한 후 약 0.4초 시뮬레이션 시간에 다시 튕겨 오르는 모습을 확인할 수 있다. 중력에 의해 떨어지면서 다시 한 번 접촉하고, 몇 초간 앞뒤로 흔들린 후 약 6초 시뮬레이션 시간에 정지 상태에 도달한다.

`Animations` 패널에서 **time**에 값을 입력하여 애니메이션의 특정 순간으로 이동할 수 있다.

재생의 장점 중 하나는 시뮬레이션 속도와 관계없이 `timeScale=1`을 설정하여 실시간 속도로 애니메이션을 볼 수 있다는 것이다. 시뮬레이션내에서 흘러가는 시간은 우리가 재생에서 보는 시뮬레이션 시간에 아무런 영향을 미치지 않는다.

## 접촉 결과를 수치적으로 리포팅하기(Report contact results numerically)

시뮬레이션 중 발생하는 접촉 결과를 보고하는 방법에 대해 알아보자. 간단한 시스템을 만들어 `MultibodyPlant`로부터 접촉 결과를 읽고 시뮬레이션 종료 시 그것들을 출력해보자.

### 접촉 결과를 publish하기 위한 간단한 시스템 사용하기(Use a simple system to publish contact results)

leaf system을 생성 관련된 정보는 [Authoring Leaf Systems](./authoring_leaf_systems.ipynb)을 참고하자.

아래에서 `ContactReporter`라는 leaf system을 정의해보자. 이 시스템은 [ContactResults.](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_contact_results.html)유형의 입력 포트를 가지고 있습니다. 강제 publish 이벤트가 발생하면 입력 포트에서 받은 접촉 결과를 출력한다. 아래 코드에서 `Publish(self, context)`을 살펴보자.

접촉 결과는 [MultibodyPlant](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_multibody_plant.html)의 출력 포트에서 get_contact_results_output_port() 메소드를 호출하여 가져올 수 있다. 사용 가능한 포트 목록은 [MultibodyPlant's input and output ports](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_multibody_plant.html#mbp_input_and_output_ports) 문서를 참조하자. 아래의 `add_contact_report()` 함수에서 먼저 이전 섹션에서 만든 `ContactReporter`를 다이어그램 빌더에 추가한다. 그런 다음, MultibodyPlant의 접촉 결과 출력 포트를 우리의 `ContactReporter`의 입력 포트에 연결한다.

이 예제 시뮬레이션은 2개 지오메트리 사이에 단 하나의 접촉 패치만 있지만, 아래 코드는 수력 탄성 물체로부터 모든 쌍의 지오메트리에서 발생하는 모든 접촉 패치를 보고할 수 있다. 하지만 ContactResults의 `point_pair_contact_info()`는 보고하지 않는다.

자세한 내용은 [ContactResults,](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_contact_results.html),  [HydroelasticContactInfo,](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_hydroelastic_contact_info.html), [ContactSurface](https://drake.mit.edu/doxygen_cxx/classdrake_1_1geometry_1_1_contact_surface.html) 문서를 참조하자.

`ContactReporter`의 `__init__()`에서 강제 publish 이벤트를 선언한다. 이 경우 사용자가 ForcedPublish()를 호출할 때만 접촉을 보고한다.(다음 섹션의 `run_simulation_with_contact_report()` 참조).
주기적 publish 이벤트를 선언하여 지정한 빈도로 접촉 결과를 보고할 수도 있다. [LeafSystem.](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html)내에서 [이벤트 주기 선언(Declare periodic events)](https://drake.mit.edu/doxygen_cxx/classdrake_1_1systems_1_1_leaf_system.html#declare_periodic_events) 부분을 참조하자.

In [ ]:
from pydrake.common.value import Value
from pydrake.multibody.plant import ContactResults
from pydrake.systems.framework import LeafSystem

class ContactReporter(LeafSystem):
    def __init__(self):
        super().__init__()  # Don't forget to initialize the base class.
        self.DeclareAbstractInputPort(
            name="contact_results",
            model_value=Value(
                # Input port will take ContactResults from MultibodyPlant
                ContactResults()))
        # Calling `ForcedPublish()` will trigger the callback.
        self.DeclareForcedPublishEvent(self.Publish)
        
    def Publish(self, context):
        print()
        print(f"ContactReporter::Publish() called at time={context.get_time()}")
        contact_results = self.get_input_port().Eval(context)
        
        num_hydroelastic_contacts = contact_results.num_hydroelastic_contacts()
        print(f"num_hydroelastic_contacts() = {num_hydroelastic_contacts}")
        
        for c in range(num_hydroelastic_contacts):
            print()
            print(f"hydroelastic_contact_info({c}): {c}-th hydroelastic contact patch")
            hydroelastic_contact_info = contact_results.hydroelastic_contact_info(c)
            
            spatial_force = hydroelastic_contact_info.F_Ac_W()
            print("F_Ac_W(): spatial force (on body A, at centroid of contact surface, in World frame) = ")
            print(f"{spatial_force}")
                        
            print("contact_surface()")
            contact_surface = hydroelastic_contact_info.contact_surface()
            num_faces = contact_surface.num_faces()
            total_area = contact_surface.total_area()
            centroid = contact_surface.centroid()
            print(f"total_area(): area of contact surface in m^2 = {total_area}")
            print(f"num_faces(): number of polygons or triangles = {num_faces}")
            print(f"centroid(): centroid (in World frame) = {centroid}")        
        
        print()

def add_contact_report(builder, plant):   
    contact_reporter = builder.AddSystem(ContactReporter())    
    builder.Connect(plant.get_contact_results_output_port(),
                    contact_reporter.get_input_port(0))
        
    return builder, plant

### 시뮬레이션 실행 및 접촉 결과 리포팅하기(Run simulation and report contact results)

아래 코드에서 보는 바와 같이, `run_simulation_with_contact_report()` 함수는 시뮬레이션이 완료된 후에 `ForcedPublish()`를 호출한다.

코드 블록의 끝 부분에서 `sim_time=0`을 사용하여 `run_simulation_with_contact_report()`를 호출한다. 시뮬레이션 time이 0일때, 출력은 순응형 박스가 강체 박스보다 훨씬 위에 위치하기 때문에 아직 접촉이 발생하지 않았다고 나온다. 따라서 다음과 같은 출력 결과를 볼 수 있다:

```
ContactReporter::Publish() called at time=0.0
num_hydroelastic_contacts() = 0
```

In [ ]:
from pydrake.systems.analysis import Simulator

def run_simulation_with_contact_report(sim_time, time_step=1e-3):
    clear_meshcat()
    
    builder, plant = add_scene(time_step)
    add_viz(builder, plant)
    add_contact_report(builder, plant)
    
    diagram = builder.Build()
    
    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.)
    
    meshcat.StartRecording(frames_per_second=256.0)
    simulator.AdvanceTo(sim_time)
    meshcat.StopRecording()

    # Forced publish after the simulation has finished.
    diagram.ForcedPublish(simulator.get_context())
    

run_simulation_with_contact_report(sim_time=0)
meshcat.PublishRecording()

이번에는 시뮬레이션 시간으로 1초 동안으로 설정하고 다시 실행해보자. 2번째 줄에서 `meshcat.PublishRecording()` 함수를 호출하기 때문에 순응형 상자가 두 번 떨어지는 것처럼 보일 수 있다. 첫 번째 애니메이션은 시뮬레이션 결과이며, 재생되는 animation은 이를 재생한 것이다. MeshCat 탭에서 확대하고 재생 속도를 느리게 하여 (`timeScale`을 0.1로 설정) 동역학 행동을 자세히 관찰하자.

아래 코드를 실행하면 토크 `tau`와 힘 `f`로 구성된 spatial force(공간 힘) F_Ac_W 를 확인할 수 있다. world 좌표계에서 표현된 힘 `f`는 대략 [-1.52, 0.89, 9.39] 로, Z축 성분은 9.39 newtons 정도이다. 이는 질량 1kg의 순응형 상자에 작용하는 중력 9.81 network보다 작다. 이는 1초간 시뮬레이션된 시간에서 순응형 상자가 여전히 앞뒤로 흔들리고 있기 때문이다.

접촉 결과는 World 프레임내에서 보고된다.

In [ ]:
run_simulation_with_contact_report(sim_time=1)
meshcat.PublishRecording()

아래 코드 블록에서는 시뮬레이션 시간을 6초 동안으로 설정하여 시뮬레이션을 실행한다. 이는 박스들이 흔들림이 거의 없는, 안정 상태에 도달하기에 충분한 시간이다. `f.Z()`는 약 9.81 newtons로 중력 힘과 거의 같게 된다.:

>   f=[-3.35e-05, -4.04e-06, 9.81]

In [ ]:
run_simulation_with_contact_report(sim_time=6)
meshcat.PublishRecording()

## 접촉 결과 시각화(Visualize contact results)

접촉 결과를 시각화하기 위해서 `ContactVisualizer`를 이전 섹션의 `Diagram`에 추가한다.

`add_contact_viz()` 함수는 `MultibodyPlant` 와 `Meshcat`를 사용하여 `ContactVisualizer`를 `DiagramBuilder`에 추가한다.

`newtons_per_meter=2e1` 설정은 각 20 뉴턴의 힘에 대해 길이 1m의 빨간색 화살표를 그린다.
`newtons_meters_per_meter=1e-1` 설정은 각 0.1 newton\*meters의 토크에 대해 길이 1m의 파란색 화살표를 그린다. 다음 섹션에서 이 시뮬레이션을 실행해보자.

In [ ]:
from pydrake.multibody.meshcat import ContactVisualizer, ContactVisualizerParams


def add_contact_viz(builder, plant):
    contact_viz = ContactVisualizer.AddToBuilder(
        builder, plant, meshcat,
        ContactVisualizerParams(
            publish_period= 1.0 / 256.0,
            newtons_per_meter= 2e1,
            newton_meters_per_meter= 1e-1))

    return builder, plant

### 접촉 시각화로 시뮬레이션 실행하기(Run simulation with contact visualization)

다음 코드는 시뮬레이션을 실행한다. MeshCat에서 빨간색 화살표는 힘 `f`를 나타내고 파란색 화살표는 토크 `tau`를 나타낸다. 힘 및 토크 벡터와 함께 접촉 패치가 움직이는 것을 볼 수 있다. 시뮬레이션이 끝나면 수치적으로 접촉 결과를 보고한다.

아래 코드는 아직 녹화를 publish하지 않았다. 다음 섹션에서 재생을 위한 녹화를 publish한다.

In [ ]:
from pydrake.systems.analysis import Simulator

def run_simulation_with_contact_report_and_viz(sim_time, time_step=1e-3):
    clear_meshcat()
    
    builder, plant = add_scene(time_step)
    add_viz(builder, plant)
    add_contact_report(builder, plant)
    add_contact_viz(builder, plant)
    
    diagram = builder.Build()
    
    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    
    meshcat.StartRecording(frames_per_second=256.0)
    simulator.AdvanceTo(sim_time)
    meshcat.StopRecording()

    # Numerically report contact results at the end of simulation.
    diagram.ForcedPublish(simulator.get_context())

    
run_simulation_with_contact_report_and_viz(sim_time=6)

### 동역학을 보기 위해 녹화한 것을 재생하기(Playback recording to appreciate dynamics)

아래 코드를 실행한 후, `timeScale = 0.1` 또는 `0.01`로 재생하여 접촉 역학을 자세히 관찰하자. 흔들리는 유연한 박스와 동기화되어 힘 및 토크 벡터가 진동하는 것을 볼 수 있다.

현재 시뮬레이션을 재생하면 접촉 힘과 모멘트가 표시되지만 접촉 패치가 적절하게 표시되지 않아 혼란스러울 수 있다. 아래 코드는 MeshCat에 publish하기 전에 녹화에서 접촉 패치를 삭제한다. Issue [19142](https://github.com/RobotLocomotion/drake/issues/19142)에서 더 자세한 설명을 확인할 수 있다.

In [ ]:
# In the current version, we can playback contact forces and torques;
# however, contact surfaces are not recorded properly.
# For now, we delete contact surfaces to prevent confusion.
# See issue https://github.com/RobotLocomotion/drake/issues/19142
meshcat.Delete("/drake/contact_forces/hydroelastic/compliant_box+rigid_box_link/contact_surface")

meshcat.PublishRecording()

## Optional exercises

다음 섹션은 선택사항이다. 여러분의 코드에 대한 이해와 더 많은 기능을 사용할 수 있도록 도와준다. 하지만 이를 배우지 않고도 수력탄성 접촉을 사용할 수 있다.

### 시뮬레이션 결과를 공유하기 위해서 html 파일로 다운받기(Download simulation result into a html file for sharing)

시뮬레이션 결과를 html 파일로 다운받을 수 있다. 이 파일로 다른 사람들이 시뮬레이션 결과를 재생해볼 수 있다. 아래 코드는 다운로드할 수 있는 URL을 출력한다. 출력된 URL을 클릭하여 다운로드한다.

In [ ]:
print(f"{meshcat.web_url()}/download")

### 처음 접촉에서 큰 충격(High impact at first contact)
0.405초에서 순응형 박스가 강체 박스에 큰 충격을 준다. 아래에서 `#`를 제거하고 코드를 실행해보자. 충돌시에 접촉 힘은 얼마인가? 대략 380 뉴턴 정도 나오는 것을 볼 수 있다.

In [ ]:
# run_simulation_with_contact_report_and_viz(sim_time=0.405)
# meshcat.Delete("/drake/contact_forces/hydroelastic/compliant_box+rigid_box_link/contact_surface")
# meshcat.PublishRecording()

### 낮은 수력탄성 계수의 효과(Effects of low hydroelastic modulus)

아래 코드는 `compliant_box_sdf`에서 수력탄성 계수의 값을 `new_hydroelastic_modulus`로 변경한다. `new_hydroelastic_modulus`의 숫자를 작은 따옴표 `'`로 묶어 문자열로 처리하는 점에 주의하자.

`new_hydroelastic_modulus`를 1e1 Pascals와 같이 비현실적으로 낮은 값으로 설정하면 순응형(유연한) 상자가 아래로 떨어지고 강체 상자를 통과하는 것을 볼 수 있다. 이는 낮은 수압탄성 계수가 중력과 일치하는 충분한 접촉력을 만들 수 없기 때문이다. 대부분의 재료는 1e7 Pa(고무)에서 1e12 Pa(탄소 나노튜브) 범위의 영률(Young's modulus)을 가지므로 수압탄성 계수는 1e5 Pa(고무)에서 1e10 Pa(탄소 나노튜브) 범위에 있어야 한다.

아래 코드에서 #을 제거하고 실행하자.

MeshCat에서 timeScale 0.1로 천천히 재생하면 약 0.41초쯤에 매우 짧은 빨간색 접촉력 화살표가 보일 것이다.

In [ ]:
# new_hydroelastic_modulus = '1e1'
# compliant_box_sdf = compliant_box(hydroelastic_modulus = new_hydroelastic_modulus)
# 
# run_simulation_with_contact_report_and_viz(sim_time=2)
# meshcat.Delete("/drake/contact_forces/hydroelastic/compliant_box+rigid_box_link/contact_surface")
# meshcat.PublishRecording()

### 높은 수력탄성 계수의 효과(Effects of high hydroelastic modulus)


마찬가지로, 아래 코드는 `new_hydroelastic_modulus`를 비현실적으로 높은 값인 1e15 Pascals로 설정한다. 유연한 상자가 각도로 튕겨져 나와 강체 상자에서 떨어지는 것을 볼 수 있다. 아래 코드에서 `#`을 제거하고 실행해보자. 비현실적으로 높은 수압탄성 계수에도 불구하고 `discrete_contact_approximation`의 선택 덕분에 시뮬레이션은 여전히 안정적이다.

In [ ]:
# new_hydroelastic_modulus = '1e15'
# compliant_box_sdf = compliant_box(hydroelastic_modulus = new_hydroelastic_modulus)
# 
# run_simulation_with_contact_report_and_viz(sim_time=3)
# meshcat.Delete("/drake/contact_forces/hydroelastic/compliant_box+rigid_box_link/contact_surface")
# meshcat.PublishRecording()

## 더 읽을꺼리

* [Hydroelastic Contact User Guide](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)

* Elandt, R., Drumwright, E., Sherman, M., & Ruina, A. (2019, November). A pressure field model for fast, robust approximation of net contact force and moment between nominally rigid objects. In 2019 IEEE/RSJ International Conference on Intelligent Robots and Systems(IROS) (pp. 8238-8245). IEEE. [link](https://arxiv.org/abs/1904.11433)

* Masterjohn, J., Guoy, D., Shepherd, J., & Castro, A. (2022). Velocity Level Approximation of Pressure Field Contact Patches. IEEE Robotics and Automation Letters 7, no. 4 (2022): 11593-11600. [link](https://arxiv.org/abs/2110.04157v2)

* Elandt, R. (2022, December). Pressure Field Contact. Dissertation. Cornell University. [link](https://ecommons.cornell.edu/handle/1813/112919)